# Spoofed Speech Detection via Maximum Likelihood Estimation of Gaussian Mixture Models
The goal of synthetic speech detection is to determine whether a speech segment $S$ is natural or synthetic/converted speeach.

This notebook implements a Gaussian mixture model maximum likelihood (GMM-ML) classifier for synthetic (spoofed) speech detection. This approach uses regular mel frequency cepstral coefficients (MFCC) features and gives the best performance on the [ASVspoof 2015 dataset](https://www.idiap.ch/dataset/avspoof) among the standard classifiers (GMM-SV, GMM-UBM, ...). For more background information see: *Hanilçi, Cemal, Tomi Kinnunen, Md Sahidullah, and Aleksandr Sizov. "Classifiers for synthetic speech detection: a comparison." In INTERSPEECH 2015*. The scripts use the Python package [Bob.Bio.SPEAR 2.04](https://pypi.python.org/pypi/bob.bio.spear/2.0.4) for speaker recogntion.

This work is part of the ["DDoS Resilient Emergency Dispatch Center"](https://www.dhs.gov/science-and-technology/news/2015/09/04/dhs-st-awards-university-houston-26m-cyber-security-research) project at the University of Houston, funded by the Department of Homeland Security (DHS).


April 19, 2015

Lorenzo Rossi

(lorenzo **[dot]** rossi **[at]** gmail **[dot]** com)

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy
import cqcc_ext
import cfcc_ext
import librosa
from cqcc import cqcc as CQCC
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

pd.set_option('display.max_rows', 300)

WAV_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/' #'ASV2019dataset/wav/' # Path to folder containing speakers .wav subfolders
LABEL_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/' #'ASV2019dataset/CM_protocol/' # Path to ground truth csv files

evalu_before = 'ASVspoof2019_LA_eval'
train_before = 'ASVspoof2019_LA_train'

EXT = '.wav'
%matplotlib inline

## Loading the Ground Truth
Load the dataframes (tables) with the labels for the training, development and evaluation (hold out) sets. Each subfolder corresponds to a different speaker. For example, T1 and D4 indicate the subfolders associated to the utterances and spoofed segments of speakers T1 and D4, respectively in training and development sets. Note that number of evaluation samples >> number of development samples >> testing samples.

You can either select the speakers in each set one by one, *e.g.*:
```
train_subfls = ['T1', 'T2']
``` 
will only load segments from speakers T1 and T2 for training,

or use all the available speakers in a certain subset by leaving the list empty, *e.g.*:
```
devel_subfls = [] 
```
will load all the available Dx speaker segments for the development stage. If you are running this notebook for the first time, you may want to start only with 2 or so speakers per set for sake of quick testing. All the scripts may take several hours to run on the full size datsets.

In [2]:
train_subfls = ['wav']#, 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T13']  #T13 used instead of T10 for gender balance
devel_subfls = []#, 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
evalu_subfls = ['wav']#, 'E2', 'E3', 'E4', 'E5', 'E6','E7',  'E8', 'E9', 'E10']
train = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.eval.balan.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
train.sort_values(['folder', 'file'], inplace=True)
print(train)
devel = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.dev.trl.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
devel.sort_values(['folder', 'file'], inplace=True)
#print(devel)
evalu = pd.read_csv(LABEL_FOLDER +'ASVspoof2019.LA.cm.30000_ubm.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
print(evalu)
#evalu.sort_values(['folder', 'file'], inplace=True)

label_2_class = {'bonafide':1, 'spoof':0}

print('training samples:',len(train))
print('development samples:',len(devel))
print('evaluation samples:',len(evalu))

        folder          file default method    source
3669   LA_0001  LA_E_1046146       -    A10     spoof
1556   LA_0001  LA_E_1168493       -    A19     spoof
2423   LA_0001  LA_E_1209085       -    A18     spoof
1759   LA_0001  LA_E_1320196       -    A10     spoof
1111   LA_0001  LA_E_1353690       -    A14     spoof
...        ...           ...     ...    ...       ...
5685   LA_0068  LA_E_8601887       -      -  bonafide
16766  LA_0068  LA_E_9127426       -      -  bonafide
26594  LA_0068  LA_E_9601956       -      -  bonafide
4371   LA_0068  LA_E_9615325       -      -  bonafide
18656  LA_0068  LA_E_9981609       -      -  bonafide

[9500 rows x 5 columns]
        folder          file default method source
0      LA_0039  LA_E_2834763       -    A11  spoof
1      LA_0014  LA_E_8877452       -    A14  spoof
2      LA_0040  LA_E_6828287       -    A16  spoof
3      LA_0022  LA_E_6977360       -    A09  spoof
4      LA_0031  LA_E_5932896       -    A13  spoof
...        ...       

## Speech Preprocessing and MFCC Extraction
Silence removal and MFCC feature extraction for training segments. More details about the bob.bio.spear involved libraries at:
https://www.idiap.ch/software/bob/docs/latest/bioidiap/bob.bio.spear/master/implemented.html

You can also skip this stage and load a set of feaures (see **Loading features** cell).

In [3]:
# Parameters
 # number of ceptral coefficients (implicit in extractor)
silence_removal_ratio = .1

In [5]:
subfolders = evalu_subfls
ground_truth = train

# initialize feature matrix
cqt = []
logcqt = []

y_list = []
#y = np.zeros((len(ground_truth),))
print("Extracting features for training stage.")

#vad = preprocessor.Energy_Thr(ratio_threshold=silence_removal_ratio)
#cepstrum = extractor.Cepstral()

k = 0
start_time = time.clock()

n_subfls = len(subfolders)
for folder in subfolders[0:n_subfls]:
    #print(folder, end=", ")
    train_folder = "".join([WAV_FOLDER,evalu_before,'/'])
    folder = "".join((train_folder,folder,'/'))
    print(folder)
    f_list = os.listdir(folder)
    # print(f_list)
    for f_name in f_list:
        #print(f_name)
        if f_name.endswith(EXT):
        # ground truth
            try:
                # 根据train中file(路径)字段，提取相应.wav文件的source(spoof or human)字段
                label = ground_truth[ground_truth.file==f_name[0:12]].source.values[0]
                #print(f_name[0:12], label)
            except IndexError:
                continue
            #y_list.append(label_2_class[label])
            # silence removal
            # print(vad)
            #rate,x = cqcc_ext.read(folder+f_name)
            #vad_data = cfcc_ext.vad(x)
            # CFCC extraction 
            #cqcc_feature = cqcc_ext.cqcc(vad_data,rate)
            #features.append(cqcc_feature)
            #k += 1
            
            
            
            y_list.append(label_2_class[label])
        # silence removal
            #print(f_name)
        # print(vad)
            x,fs = librosa.load(folder+f_name, sr = 16000)
            #x = x.reshape(x.shape[0], 1)
            #print(x)
            B = 96
            fmax = fs/2
            fmin = fmax/2**9
            d = 16
            cf = 19
            ZsdD = 'ZsdD'

            # MFCC extraction 
            #CQcc, LogP_absCQT, TimeVec, FreqVec, Ures_LogP_absCQT, Ures_FreqVec, absCQT = CQCC(x, fs, B, fmax, fmin, d, cf, ZsdD)
            #print(CQcc)
            #print("????????????")
            C = np.abs(librosa.cqt(x, sr=16000, fmin=librosa.note_to_hz('C2'),n_bins=60))
            cqt.append(np.transpose(C))
            #cqcc_orig.append(CQcc[:, :20])
            #cqcc_one.append(CQcc[:, 20:40])
            #cqcc_two.append(CQcc[:, 40:60])
            #cqcc_orig_one.append(CQcc[:, :40])
            #print(features[:, :40].shape)
            #print("done")

# 这里是将人声与合成声进行了区分的，根据就是y
#y = np.array(y_list)
#Xf_cqcc = np.array(cqcc)
#Xf_cqcc_orig = np.array(cqcc_orig)
#Xf_cqcc_one = np.array(cqcc_one)
#Xf_cqcc_two = np.array(cqcc_two)
#Xf_cqcc_orig_one = np.array(cqcc_orig_one)
#print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")
#np.save('wavenet_Xf_from_trainset_500.npy',Xf_cqcc)
#np.save('Xf_cqcc_orig.npy',Xf_cqcc_orig)
#np.save('Xf_cqcc_one.npy',Xf_cqcc_one)
#np.save('Xf_cqcc_two.npy',Xf_cqcc_two)
#np.save('Xf_cqcc_orig_one.npy',Xf_cqcc_orig_one)
#np.save('y_cqcc.npy',y_list)
#print('Feature and label matrices saved to disk')

Extracting features for training stage.
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_eval/wav/


/home/wujunting/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


In [6]:
y = np.array(y_list)
cqt = np.array(cqt)
#Xf_cqcc_orig = np.array(cqcc_orig)
#print(Xf_cqcc.shape)
#Xf_cqcc_one = np.array(cqcc_one)
#print(Xf_cqcc.shape)
#Xf_cqcc_two = np.array(cqcc_two)
#print(Xf_cqcc.shape)
#Xf_cqcc_orig_one = np.array(cqcc_orig_one)
#print(Xf_cqcc.shape)
np.save('wavenet_cqt_from_evalset_balan.npy',cqt)
np.save('wavenet_y_from_evalset_balan.npy',y_list)
#print("done1")
#np.save('Xf_cqcc_orig.npy',Xf_cqcc_orig)
#print("done2")
#np.save('Xf_cqcc_one.npy',Xf_cqcc_one)
#print("done3")
#np.save('Xf_cqcc_two.npy',Xf_cqcc_two)
#print("done4")
#np.save('Xf_cqcc_orig_one.npy',Xf_cqcc_orig_one)
#print("done5")
#np.save('wavenet_y_from_evalset_balan.npy',y_list)
print('Feature and label matrices saved to disk')

/home/wujunting/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Feature and label matrices saved to disk


In [4]:
import numpy as np
cqcc = np.load('Xf_ubm_cqcc.npy')
from sklearn.mixture import GaussianMixture
print(cqcc.shape)
gmm=GaussianMixture(n_components=512,covariance_type='diag',max_iter=200,n_init=1)
print("done")

(3671692, 84)
done


In [ ]:
gmm.fit(cqcc)
print("done")

In [34]:
np.vstack(Xf[y==1]).shape

(862811, 60)

In [80]:
ground_truth

,folder,file,default,method,source,z,z_gmm,score_gmm
107206,E2,E200001s,-,spoof,spoof,0,1.0,7.622108
98412,E2,E200002s,-,spoof,spoof,0,1.0,10.544859
102443,E2,E200003h,-,bonafide,bonafide,1,0.0,-2.888099
103712,E2,E200004s,-,spoof,spoof,0,0.0,-2.797664
98984,E2,E200005h,-,bonafide,bonafide,1,1.0,17.829845
...,...,...,...,...,...,...,...,...
105581,E2,E208996h,-,bonafide,bonafide,1,0.0,-3.521843
100661,E2,E208997h,-,bonafide,bonafide,1,0.0,-3.759985
106825,E2,E208998s,-,spoof,spoof,0,1.0,6.707735
102766,E2,E208999h,-,bonafide,bonafide,1,1.0,7.150391


### Saving features

In [18]:
np.save('Xf_cqcc.npy',Xf_mfcc)
np.save('Xf_cqcc_orig.npy',Xf_mfcc_orig)
np.save('Xf_cqcc_one.npy',Xf_mfcc_one)
np.save('Xf_cqcc_two.npy',Xf_mfcc_two)
np.save('Xf_cqcc_orig_one.npy',Xf_mfcc_orig_one)
np.save('y_cqcc.npy',y)
print('Feature and label matrices saved to disk')

Feature and label matrices saved to disk


### Loading features

In [20]:
# Load already extracter features to skip the preprocessing-extraction stage
# Xf = np.load('train_features_10.npy')
# y = np.load('y_10.npy')
Xf = np.load('Xf_cqcc.npy',allow_pickle=True)
y = np.load('y_cqcc.npy',allow_pickle=True)

# GMM - ML Classification

In [22]:
print(Xf.shape)

(25380,)


## GMM Training
Train the GMMs for natural and synthetic speach. For documentation on bob.bio k-means and GMM machines see:
https://pythonhosted.org/bob.learn.em/guide.html

You can also skip the training stage and load an already trained GMM model (see cell **Loading GMM Model**).

In [15]:
# Parameters of the GMM machines
n_gaussians = 512 # number of Gaussians
max_iterats = 25 # maximum number of iterations
n_ceps = 60

### GMM for natural speech 

In [16]:
# Initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_nat = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_nat.shape,Xf[y==1].shape)
em.train(kmeansTrainer, kmeans_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#kmeans_nat.means

# initialize and train GMM machine
gmm_nat = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_nat.means = kmeans_nat.means
em.train(trainer, gmm_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
print(gmm_nat)
gmm_nat.save(HDF5File('pzb_cqcc_nat.hdf5', 'w'))

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(512, 60) (2580,)


/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Done in: 100.58375983333332 minutes


### GMM for synthetic speech

In [5]:
# initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_synt = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_synt.shape,Xf[y==0].shape)
em.train(kmeansTrainer, kmeans_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)

# initialize and train GMM machine
gmm_synt = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_synt.means = kmeans_synt.means
em.train(trainer, gmm_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
gmm_synt.save(HDF5File('pzb_cqcc_synt.hdf5', 'w'))
print(gmm_synt)

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


NameError: name 'n_gaussians' is not defined

### Saving GMM model

In [52]:
gmm_nat.save(HDF5File('gmm_nat_500seg.hdf5', 'w'))
gmm_synt.save(HDF5File('gmm_synt_500seg.hdf5', 'w'))

### Loading GMM model

In [3]:
gmm_nat = em.GMMMachine()
gmm_nat.load(HDF5File('pzb_cqcc_nat.hdf5', 'r'))
gmm_synt = em.GMMMachine()
gmm_synt.load(HDF5File('pzb_cqcc_synt.hdf5','r'))
gmm_orig_nat = em.GMMMachine()
gmm_orig_nat.load(HDF5File('pzb_cqcc_orig_nat.hdf5', 'r'))
gmm_orig_synt = em.GMMMachine()
gmm_orig_synt.load(HDF5File('pzb_cqcc_orig_synt.hdf5','r'))
gmm_one_nat = em.GMMMachine()
gmm_one_nat.load(HDF5File('pzb_cqcc_one_nat.hdf5', 'r'))
gmm_one_synt = em.GMMMachine()
gmm_one_synt.load(HDF5File('pzb_cqcc_one_synt.hdf5','r'))
gmm_two_nat = em.GMMMachine()
gmm_two_nat.load(HDF5File('pzb_cqcc_two_nat.hdf5', 'r'))
gmm_two_synt = em.GMMMachine()
gmm_two_synt.load(HDF5File('pzb_cqcc_two_synt.hdf5','r'))
gmm_orig_one_nat = em.GMMMachine()
gmm_orig_one_nat.load(HDF5File('pzb_cqcc_orig_one_nat.hdf5', 'r'))
gmm_orig_one_synt = em.GMMMachine()
gmm_orig_one_synt.load(HDF5File('pzb_cqcc_orig_one_synt.hdf5','r'))

## GMM-ML Scoring
Extract the features for the testing data, compute the likelihood ratio test and  compute ROC AUC and estimated EER scores.

In [6]:
subfolders = train_subfls
ground_truth = train

print(len(ground_truth))
# initialize score and class arrays
gmm_nat_score = np.zeros(len(ground_truth),)
gmm_synt_score = np.zeros(len(ground_truth),)
llr_gmm_score = np.zeros(len(ground_truth),)
z_gmm = np.zeros(len(ground_truth),)

gmm_nat_orig_score = np.zeros(len(ground_truth),)
gmm_synt_orig_score = np.zeros(len(ground_truth),)
llr_gmm_orig_score = np.zeros(len(ground_truth),)
z_orig_gmm = np.zeros(len(ground_truth),)

gmm_nat_one_score = np.zeros(len(ground_truth),)
gmm_synt_one_score = np.zeros(len(ground_truth),)
llr_gmm_one_score = np.zeros(len(ground_truth),)
z_one_gmm = np.zeros(len(ground_truth),)

gmm_nat_two_score = np.zeros(len(ground_truth),)
gmm_synt_two_score = np.zeros(len(ground_truth),)
llr_gmm_two_score = np.zeros(len(ground_truth),)
z_two_gmm = np.zeros(len(ground_truth),)

gmm_nat_orig_one_score = np.zeros(len(ground_truth),)
gmm_synt_orig_one_score = np.zeros(len(ground_truth),)
llr_gmm_orig_one_score = np.zeros(len(ground_truth),)
z_orig_one_gmm = np.zeros(len(ground_truth),)
print(status)

vad = preprocessor.Energy_Thr(ratio_threshold=.1)
cepstrum = extractor.Cepstral()

k = 0
thr = .5

#speaker = ground_truth[ground_truth.folder==speaker_id]
f_list = list(ground_truth.file)
evalu_folder = "".join([WAV_FOLDER,train_before,'/'])
folder = "".join([evalu_folder,evalu_subfls[0],'/'])
#print(f_list)
print(folder)
for f in f_list:
    if isinstance(f,str):
        f_name = "".join([folder,f,EXT])
    if os.path.exists(f_name):
        # silence removal
        # print(f_name)
        # print(vad)
        x,fs = librosa.load(f_name, sr = 16000)
        x = x.reshape(x.shape[0], 1)
        print(f_name)
        B = 96
        fmax = fs/2
        fmin = fmax/2**9
        d = 16
        cf = 19
        ZsdD = 'ZsdD'
        CQcc, LogP_absCQT, TimeVec, FreqVec, Ures_LogP_absCQT, Ures_FreqVec, absCQT = CQCC(x, fs, B, fmax, fmin, d, cf, ZsdD)
        print("orig_shape:", CQcc.shape)
        if CQcc.shape[0] >= 400:
            CQcc = CQcc[:400, :]
        else:
            CQcc_orig = CQcc
            make_up = int(400/CQcc.shape[0])
            for i in range(make_up-1):
                #print(i)
                CQcc = np.vstack((CQcc,CQcc_orig))
            CQcc = np.vstack((CQcc, CQcc_orig[:(400-(CQcc_orig.shape[0]*make_up)), :]))
        print("??????")
        print(CQcc.shape)
        # Log likelihood ratio computation
        gmm_nat_score[k] = gmm_nat(CQcc)
        #print(gmm_nat_score[k])
        gmm_synt_score[k] = gmm_synt(CQcc)
        #print(gmm_synt_score[k])
        llr_gmm_score[k] = gmm_nat_score[k] - gmm_synt_score[k]
        z_gmm[k] = int(llr_gmm_score[k]>0)
        
        gmm_nat_orig_score[k] = gmm_orig_nat(CQcc[: , :20])
        gmm_synt_orig_score[k] = gmm_orig_synt(CQcc[: , :20])
        llr_gmm_orig_score[k] = gmm_nat_orig_score[k] - gmm_synt_orig_score[k]
        z_orig_gmm[k] = int(llr_gmm_orig_score[k]>0)        
        
        gmm_nat_one_score[k] = gmm_one_nat(CQcc[: , 20:40])
        gmm_synt_one_score[k] = gmm_one_synt(CQcc[: , 20:40])
        llr_gmm_one_score[k] = gmm_nat_one_score[k] - gmm_synt_one_score[k]
        z_one_gmm[k] = int(llr_gmm_one_score[k]>0) 
        
        gmm_nat_two_score[k] = gmm_two_nat(CQcc[: , 40:])
        gmm_synt_two_score[k] = gmm_two_synt(CQcc[: , 40:])
        llr_gmm_two_score[k] = gmm_nat_two_score[k] - gmm_synt_two_score[k]
        z_two_gmm[k] = int(llr_gmm_two_score[k]>0)       
        
        gmm_nat_orig_one_score[k] = gmm_orig_one_nat(CQcc[: , :40])
        gmm_synt_orig_one_score[k] = gmm_orig_one_synt(CQcc[: , :40])
        llr_gmm_orig_one_score[k] = gmm_nat_orig_one_score[k] - gmm_synt_orig_one_score[k]
        z_orig_one_gmm[k] = int(llr_gmm_orig_one_score[k]>0)
        #class_2_label={0:'spoof',1:'bonafide'}
        #if label_2_class[ground_truth.iloc[k].source] != z_gmm[k]:
        #    F_FILE = np.append(F_FILE,ground_truth.iloc[k].file)
            
    k += 1
    

ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
ground_truth['z_gmm'] = z_gmm
ground_truth['score_gmm'] = llr_gmm_score
ground_truth['gmm_nat_score'] = gmm_nat_score
ground_truth['gmm_synt_score'] = gmm_synt_score
#print(roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

25380
train
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1004644.wav
orig_shape: (225, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1006715.wav
orig_shape: (266, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1007408.wav
orig_shape: (381, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1017386.wav
orig_shape: (283, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1025525.wav
orig_shape: (673, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1029621.wav
orig_shape: (750, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1030843.wav
orig_shape: (367, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1033159.wav
orig_shape:

orig_shape: (533, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1477244.wav
orig_shape: (800, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1499248.wav
orig_shape: (454, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1511522.wav
orig_shape: (698, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1519590.wav
orig_shape: (680, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1525866.wav
orig_shape: (379, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1531370.wav
orig_shape: (531, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1541983.wav
orig_shape: (633, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_1551351.wav
orig_shape: (343, 60)
??????
(400, 60)
/home/wuj

orig_shape: (683, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2042929.wav
orig_shape: (594, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2048941.wav
orig_shape: (487, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2051110.wav
orig_shape: (294, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2056710.wav
orig_shape: (394, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2057403.wav
orig_shape: (196, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2058789.wav
orig_shape: (445, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2086386.wav
orig_shape: (180, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2099563.wav
orig_shape: (171, 60)
??????
(400, 60)
/home/wuj

orig_shape: (320, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2488117.wav
orig_shape: (450, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2488195.wav
orig_shape: (454, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2494677.wav
orig_shape: (317, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2495731.wav
orig_shape: (735, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2503173.wav
orig_shape: (342, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2518848.wav
orig_shape: (528, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2520083.wav
orig_shape: (370, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2534971.wav
orig_shape: (379, 60)
??????
(400, 60)
/home/wuj

orig_shape: (474, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3051141.wav
orig_shape: (364, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3055656.wav
orig_shape: (554, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3055700.wav
orig_shape: (290, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3056559.wav
orig_shape: (257, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3057409.wav
orig_shape: (505, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3060458.wav
orig_shape: (242, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3061594.wav
orig_shape: (285, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3067843.wav
orig_shape: (565, 60)
??????
(400, 60)
/home/wuj

...

orig_shape: (458, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2705905.wav
orig_shape: (234, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2727451.wav
orig_shape: (813, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2745780.wav
orig_shape: (300, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2747022.wav
orig_shape: (385, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2748681.wav
orig_shape: (734, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2762685.wav
orig_shape: (428, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2766626.wav
orig_shape: (729, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_2772197.wav
orig_shape: (669, 60)
??????
(400, 60)
/home/wuj

orig_shape: (268, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3258161.wav
orig_shape: (509, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3265392.wav
orig_shape: (624, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3266022.wav
orig_shape: (420, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3267673.wav
orig_shape: (479, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3279709.wav
orig_shape: (440, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3282702.wav
orig_shape: (334, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3286362.wav
orig_shape: (533, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3286480.wav
orig_shape: (634, 60)
??????
(400, 60)
/home/wuj

orig_shape: (230, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3744884.wav
orig_shape: (734, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3745143.wav
orig_shape: (385, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3747538.wav
orig_shape: (446, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3748725.wav
orig_shape: (278, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3751646.wav
orig_shape: (320, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3768404.wav
orig_shape: (256, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3769139.wav
orig_shape: (404, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_3778918.wav
orig_shape: (819, 60)
??????
(400, 60)
/home/wuj

orig_shape: (527, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4177986.wav
orig_shape: (388, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4180224.wav
orig_shape: (655, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4183341.wav
orig_shape: (465, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4184798.wav
orig_shape: (318, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4187934.wav
orig_shape: (283, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4197596.wav
orig_shape: (225, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4200125.wav
orig_shape: (415, 60)
??????
(400, 60)
/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/wav/LA_T_4205673.wav
orig_shape: (356, 60)
??????
(400, 60)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
np.save('./score/400_score_train_nat.npy',gmm_nat_score)
np.save('./score/400_score_train_synt.npy',gmm_synt_score)

np.save('./score/400_score_train_orig_nat.npy',gmm_nat_orig_score)
np.save('./score/400_score_train_orig_synt.npy',gmm_synt_orig_score)

np.save('./score/400_score_train_one_nat.npy',gmm_nat_one_score)
np.save('./score/400_score_train_one_synt.npy',gmm_synt_one_score)

np.save('./score/400_score_train_two_nat.npy',gmm_nat_two_score)
np.save('./score/400_score_train_two_synt.npy',gmm_synt_two_score)

np.save('./score/400_score_train_orig_one_nat.npy',gmm_nat_orig_one_score)
np.save('./score/400_score_train_orig_one_synt.npy',gmm_synt_orig_one_score)


In [10]:
ground_truth = train
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
print(ground_truth['z'].shape)
np.save('./score/z_train.npy', ground_truth['z'])

(25380,)


In [12]:
# Performance evaluation
ground_truth = train
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
ground_truth['z_gmm'] = z_gmm


#实际humans个数
humans = sum(ground_truth.z==1)
print("实际humans个数:", humans)

#实际spoofed个数
spoofed = sum(ground_truth.z==0)
print("实际spoofed个数:", spoofed)

#预测humans个数
pre_humans = sum(ground_truth.z_gmm==1)
print("预测humans个数:", pre_humans)

#预测spoofed个数
pre_spoofed = sum(ground_truth.z_gmm==0)
print("预测spoofed个数:", pre_spoofed)

tp = 0
tn = 0
fp = 0
fn = 0
for i in ground_truth.index:
    if ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==1:
        tp = tp + 1
    elif ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==0:
        fn = fn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==0:
        tn = tn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==1:
        fp = fp + 1
print("TP_Num:", tp)
print("TN_Num:", tn)
print("FP_Num:", fp)
print("FN_Num:", fn)

tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(fp+tn)
fnr = fn/(tp+fn)
acc = (tp+tn)/(tp+fp+tn+fn)
eer = (fpr*fnr)/(fpr+fnr)
print("TPR:", tpr)
print("FPR:", fpr)
print("TNR:", tnr)
print("FNR:", fnr)
print("ROC:", roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print("Accuracy:", acc)
print("EER:", eer)

# humans = z_gmm[z_dvl==0]
# spoofed = z_gmm[z_dvl==1]
# fnr = 100*(1-(humans<thr).sum()/len(humans))
# fpr = 100*(1-(spoofed>=thr).sum()/len(spoofed))
# print("ROC AUC score:", roc_auc_score(z_dvl,z_gmm))
# print("False negative rate %:", fnr)
# print("False positive rate %:", fpr)
# print("EER %: <=", (fnr+fpr)/2)

实际humans个数: 2580
实际spoofed个数: 22800
预测humans个数: 7324
预测spoofed个数: 18056
TP_Num: 1370
TN_Num: 16846
FP_Num: 5954
FN_Num: 1210
TPR: 0.5310077519379846
FPR: 0.261140350877193
TNR: 0.738859649122807
FNR: 0.4689922480620155
ROC: 0.6349337005303958
Accuracy: 0.7177304964539007
EER: 0.16774049041987163


In [62]:
method = {}
for index, row in ground_truth.iterrows():
    if row['z'] != row['z_gmm']:
        if row['method'] in method:
            method[row['method']] = method[row['method']] + 1
        else:
            method[row['method']] = 1
print(method)

{'-': 6757, 'A07': 17, 'A16': 18, 'A10': 28, 'A12': 21, 'A11': 25, 'A13': 6, 'A15': 4, 'A18': 13, 'A17': 14, 'A19': 39, 'A08': 2}


In [47]:
ground_truth

,folder,file,method,source,z,z_gmm,score_gmm
2597,D1,D1_1002598,human,human,1,1.0,1.447147
2598,D1,D1_1002599,human,human,1,1.0,2.336380
2599,D1,D1_1002600,human,human,1,1.0,3.082637
2600,D1,D1_1002601,human,human,1,1.0,3.716948
2601,D1,D1_1002602,human,human,1,1.0,0.914256
...,...,...,...,...,...,...,...
7767,D3,D3_1007768,S1,spoof,0,0.0,-2.641017
7768,D3,D3_1007769,S2,spoof,0,0.0,-0.571958
7769,D3,D3_1007770,S3,spoof,0,0.0,-5.441518
7770,D3,D3_1007771,S4,spoof,0,0.0,-4.762759


### EER computation

Adjust the threshold $thr$ to reduce $FNR-FPR$ for a more accurate estimate of the $EER$.

The Equal Error Rate ($EER$) is the value where the false negative rate ($FNR$) equals the false positive rate ($FPR$). It's an error metric commonly used to characterize biometric systems.

In [47]:
thr = -.115
pz = llr_gmm_score
spoofed = pz[np.array(ground_truth.z)==1]
humans = pz[np.array(ground_truth.z)==0]
fnr = 100*(humans>thr).sum()/len(humans)
fpr = 100*(spoofed<=thr).sum()/len(spoofed)
print("False negative vs positive rates %:", fnr, fpr)
print("FNR - FPR %:", fnr-fpr)
if np.abs(fnr-fpr) <.25:
    print("EER =", (fnr+fpr)/2,"%")
else:
    print("EER ~", (fnr+fpr)/2,"%")

False negative vs positive rates %: 0.3345997002544352 99.53318745441284
FNR - FPR %: -99.19858775415841
EER ~ 49.933893577333635 %
